In [1]:
import torch
from transformers.file_utils import is_tf_available, is_torch_available, is_torch_tpu_available
from transformers import BertTokenizerFast, BertForSequenceClassification
from transformers import Trainer, TrainingArguments
from transformers import AutoModel, AutoTokenizer
import numpy as np
import random
from sklearn.model_selection import train_test_split
import pandas as pd
import pickle
def set_seed(seed: int):
    """
    Helper function for reproducible behavior to set the seed in ``random``, ``numpy``, ``torch`` and/or ``tf`` (if
    installed).

    Args:
        seed (:obj:`int`): The seed to set.
    """
    random.seed(seed)
    np.random.seed(seed)
    if is_torch_available():
        torch.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        # ^^ safe to call this function even if cuda is not available
    if is_tf_available():
        import tensorflow as tf

        tf.random.set_seed(seed)

set_seed(1)

## Model preparation

In [2]:
model_name = "bert-base-chinese"#bert-base-chinese bert-base-uncased
tokenizer = AutoTokenizer.from_pretrained(model_name)

## Data Processing

In [3]:
lang = 'chinese'

### big model data

In [113]:
#df = pd.read_csv('twitter_normal_data.csv', header = None)

import pickle
with open('comment_final.pickle', 'rb') as f:
    df = pickle.load(f)
    df.columns = [0,1]
emojis = df[1].unique()
for i in range(len(emojis)):
    if len(df[df[1] == emojis[i]]) <= 10:
        df = df.drop(df[df[1] == emojis[i]].index, axis = 0)
emoji_dict = {}
emojis = df[1].unique()
for i in range(len(emojis)):
    emoji_dict[emojis[i]] = i
tokenizer.add_tokens(list(emojis))
tokenizer.save_pretrained(lang + '/tokenizer')

df[1] = df[1].replace(emoji_dict)
with open(lang + '/emoji_dict.pickle', 'wb') as f:
     pickle.dump(emoji_dict,f)

tokenizer config file saved in chinese/tokenizer\tokenizer_config.json
Special tokens file saved in chinese/tokenizer\special_tokens_map.json


### customized model data

In [5]:
df = pd.read_csv('all.csv')
df.columns = [0,1,2]

with open(lang +'/emoji_dict.pickle', 'rb') as f:
     emoji_dict = pickle.load(f)
df[1] = df[1].replace(emoji_dict)
df  = df.dropna()

In [6]:
for i in range(len(emoji_dict)):
    if len(df[df[1] == i]) == 1:
        df = df.drop(df[df[1] == i].index, axis = 0)
    if len(df[df[1] == i]) == 0:
        df = df.append(pd.DataFrame([['',i,''],['',i,'']]), ignore_index=True)
df = df[df[1].str.isnumeric().fillna(True)]

### pretrained model

In [101]:
model_name = './chinese/original-19500'#./chinese/customize-14500
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=319).to("cuda")#len(df[1].unique())
tokenizer = AutoTokenizer.from_pretrained(lang + '/tokenizer')
#model.resize_token_embeddings(len(tokenizer))

loading configuration file ./chinese/original-19500\config.json
Model config BertConfig {
  "_name_or_path": "checkpoint-15500",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20",
    "21": "LABEL_21",
    "22": "LABEL_22",
    "23": "LABEL_23",
    "24": "LABEL_24",
    "25": "LABEL_25",
    "26": "LABEL_26",
    "27": "LABEL_27",
    "28": "LABEL_28",
  

loading weights file ./chinese/original-19500\pytorch_model.bin
All model checkpoint weights were used when initializing BertForSequenceClassification.

All the weights of BertForSequenceClassification were initialized from the model checkpoint at ./chinese/original-19500.
If your task is similar to the task the model of the checkpoint was trained on, you can already use BertForSequenceClassification for predictions without further training.
loading file chinese/tokenizer\vocab.txt
loading file chinese/tokenizer\tokenizer.json
loading file chinese/tokenizer\added_tokens.json
loading file chinese/tokenizer\special_tokens_map.json
loading file chinese/tokenizer\tokenizer_config.json


In [114]:
train_text, test_text, trainY, testY = train_test_split(df[0].values, df[1].values, test_size=0.2, shuffle = True, stratify = df[1])

In [115]:
trainX = tokenizer(train_text.tolist(), truncation=True, padding=True, max_length=512)
testX = tokenizer(test_text.tolist(), truncation=True, padding=True, max_length=512)

In [116]:
class ReviewDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        item["labels"] = torch.tensor([self.labels[idx]], dtype=torch.long)
        return item

    def __len__(self):
        return len(self.labels)

# convert our tokenized data into a torch Dataset

train_dataset = ReviewDataset(trainX, trainY)
test_dataset = ReviewDataset(testX, testY)

In [109]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    # calculate accuracy using sklearn's function
    acc = accuracy_score(labels, preds)
    macro = f1_score(labels, preds, average='macro')
    micro = f1_score(labels, preds, average='micro')
    weighted = f1_score(labels, preds, average='weighted')
    k = 5
    preds_k = torch.topk(torch.tensor(pred.predictions), k).indices.cpu().numpy()
    c=0
    for i,l in enumerate(labels):
        if l in preds_k[i]:
            c += 1
    return {
      'accuracy': acc,
        'macro': macro,
        'micro': micro,
        'weighted': weighted,
        'acc_at_k': c/len(labels)
    }

In [117]:
training_args = TrainingArguments(
    output_dir='D:/reviewsData/result/chinese/customized',          # output directory
    num_train_epochs=9,              # total number of training epochs
    per_device_train_batch_size=4,  # batch size per device during training
    per_device_eval_batch_size=4,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    load_best_model_at_end=True,     # load the best model when finished training (default metric is loss)
    # but you can specify `metric_for_best_model` argument to change to accuracy or other metric
    logging_steps=500,               # log & save weights each logging_steps
    evaluation_strategy="steps",     # evaluate each `logging_steps`
)

using `logging_steps` to initialize `eval_steps` to 500
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [118]:
trainer = Trainer(
    model=model,                         # the instantiated Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=test_dataset,          # evaluation dataset
    compute_metrics=compute_metrics,     # the callback that computes metrics of interest
)

In [ ]:
trainer.train()

***** Running training *****
  Num examples = 6811
  Num Epochs = 9
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 15327


Step,Training Loss,Validation Loss,Accuracy,Macro,Micro,Weighted
500,0.436300,2.791839,0.604815,0.417247,0.604815,0.592545
1000,0.719200,2.813612,0.595420,0.407451,0.595420,0.585283
1500,0.753200,2.830524,0.598943,0.410634,0.598943,0.584184
2000,0.729100,2.774474,0.593658,0.408865,0.593658,0.580663
2500,0.731700,2.802470,0.596007,0.403153,0.596007,0.584797
3000,0.673400,2.879263,0.597769,0.409481,0.597769,0.588098
3500,0.657700,2.879191,0.599530,0.418100,0.599530,0.588382
4000,0.572900,2.808121,0.603641,0.427583,0.603641,0.598950
4500,0.649400,2.896103,0.601292,0.406254,0.601292,0.588419
5000,0.627500,2.944963,0.596594,0.403492,0.596594,0.583616


***** Running Evaluation *****
  Num examples = 1703
  Batch size = 4
Saving model checkpoint to D:/reviewsData/result/chinese/customized\checkpoint-500
Configuration saved in D:/reviewsData/result/chinese/customized\checkpoint-500\config.json
Model weights saved in D:/reviewsData/result/chinese/customized\checkpoint-500\pytorch_model.bin
***** Running Evaluation *****
  Num examples = 1703
  Batch size = 4
Saving model checkpoint to D:/reviewsData/result/chinese/customized\checkpoint-1000
Configuration saved in D:/reviewsData/result/chinese/customized\checkpoint-1000\config.json
Model weights saved in D:/reviewsData/result/chinese/customized\checkpoint-1000\pytorch_model.bin
***** Running Evaluation *****
  Num examples = 1703
  Batch size = 4
Saving model checkpoint to D:/reviewsData/result/chinese/customized\checkpoint-1500
Configuration saved in D:/reviewsData/result/chinese/customized\checkpoint-1500\config.json
Model weights saved in D:/reviewsData/result/chinese/customized\check

***** Running Evaluation *****
  Num examples = 1703
  Batch size = 4
Saving model checkpoint to D:/reviewsData/result/chinese/customized\checkpoint-12500
Configuration saved in D:/reviewsData/result/chinese/customized\checkpoint-12500\config.json
Model weights saved in D:/reviewsData/result/chinese/customized\checkpoint-12500\pytorch_model.bin
***** Running Evaluation *****
  Num examples = 1703
  Batch size = 4
Saving model checkpoint to D:/reviewsData/result/chinese/customized\checkpoint-13000
Configuration saved in D:/reviewsData/result/chinese/customized\checkpoint-13000\config.json
Model weights saved in D:/reviewsData/result/chinese/customized\checkpoint-13000\pytorch_model.bin
***** Running Evaluation *****
  Num examples = 1703
  Batch size = 4
Saving model checkpoint to D:/reviewsData/result/chinese/customized\checkpoint-13500
Configuration saved in D:/reviewsData/result/chinese/customized\checkpoint-13500\config.json
Model weights saved in D:/reviewsData/result/chinese/custo

In [119]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 6659
  Batch size = 4


{'eval_loss': 1.2417843341827393,
 'eval_accuracy': 0.709866346298243,
 'eval_macro': 0.5921293629851295,
 'eval_micro': 0.709866346298243,
 'eval_weighted': 0.6974557179800941,
 'eval_acc_at_k': 0.8720528607899084,
 'eval_runtime': 56.9387,
 'eval_samples_per_second': 116.95,
 'eval_steps_per_second': 29.242}

In [ ]:
training_args = torch.load('checkpoint-19500/training_args.bin')
training_args

In [ ]:
def get_prediction(text):
    # prepare our text into tokenized sequence
    inputs = tokenizer(text, padding=True, truncation=True, max_length=max_length, return_tensors="pt").to("cuda")
    # perform inference to our model
    outputs = model(**inputs)
    # get output probabilities by doing softmax
    probs = outputs[0].softmax(1)
    # executing argmax function to get the candidate label
    return target_names[probs.argmax()]

In [ ]:
# Example #1
text = """
The first thing is first. 
If you purchase a Macbook, you should not encounter performance issues that will prevent you from learning to code efficiently.
However, in the off chance that you have to deal with a slow computer, you will need to make some adjustments. 
Having too many background apps running in the background is one of the most common causes. 
The same can be said about a lack of drive storage. 
For that, it helps if you uninstall xcode and other unnecessary applications, as well as temporary system junk like caches and old backups.
"""
print(get_prediction(text))